In [1]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
from pandas_datareader import data as pdr
yf.pdr_override() # <== that's all it takes :-)
import numpy as np
import statistics
from pprint import pprint
from math import isnan

C:\Users\Yingchen\Anaconda3\envs\finance\lib\site-packages\pandas_datareader\compat\__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


In [2]:
sp500_history1 = pd.read_csv('./sp500_history.csv')
sp500_history1['date'] = pd.to_datetime(sp500_history1['date']) 
sp500_history1 = sp500_history1[['date', 'value', 'name', 'variable']]

start_date = datetime.strptime('2016-1-1', '%Y-%m-%d')
end_date = datetime.strptime('2020-12-31', '%Y-%m-%d')

# greater than the start date and smaller than the end date
mask = (sp500_history1['date'] > start_date) & (sp500_history1['date'] <= end_date)
sp500_history1 = sp500_history1.loc[mask]

# Sort
sp500_history1.sort_values(['date', 'variable'], ascending=[True, True])

# Rename columns
sp500_history1.columns = ['effective_date', 'ticker', 'wiki_name', 'type']

sp500_history1 = sp500_history1.drop_duplicates()
sp500_history1 = sp500_history1.reset_index(drop=True)
sp500_history1 = sp500_history1.replace(to_replace = 'added_ticker', value = 'ADDED')
sp500_history1 = sp500_history1.replace(to_replace = 'removed_ticker', value = 'DELETED')
sp500_history1

,effective_date,ticker,wiki_name,type
0,2016-01-05,FOSL,Fossil Group,DELETED
1,2016-01-05,WLTW,Willis Towers Watson,ADDED
2,2016-01-19,EXR,Extra Space Storage,ADDED
3,2016-01-19,CB,Chubb Corp,DELETED
4,2016-02-01,FRT,Federal Realty Investment Trust,ADDED
...,...,...,...,...
190,2019-12-23,LYV,Live Nation Entertainment,ADDED
191,2019-12-23,MAC,Macerich,DELETED
192,2019-12-23,TRIP,TripAdvisor,DELETED
193,2019-12-23,ZBRA,Zebra Technologies,ADDED


In [23]:
sp500_history2 = pd.read_excel('./sp500_rebalance_announcements.xlsx')
sp500_history2['implementation_date'] = pd.to_datetime(sp500_history2['implementation_date']) 
sp500_history2['effective_date'] = pd.to_datetime(sp500_history2['effective_date']) 
sp500_history2['announcement_date'] = pd.to_datetime(sp500_history2['announcement_date']) 
sp500_history2 = sp500_history2[['announcement_date', 'implementation_date', 'effective_date', 'ticker', 'name', 'type']]

start_date = datetime.strptime('2015-12-20', '%Y-%m-%d')
end_date = datetime.strptime('2020-12-31', '%Y-%m-%d')

# greater than the start date and smaller than the end date
mask = (sp500_history2['announcement_date'] > start_date) & (sp500_history2['announcement_date'] <= end_date)
sp500_history2 = sp500_history2.loc[mask]

# Sort
sp500_history2 = sp500_history2.sort_values(['announcement_date', 'type'], ascending=[True, True])
sp500_history2 = sp500_history2.drop_duplicates()
sp500_history2 = sp500_history2.reset_index(drop=True)
sp500_history2

,announcement_date,implementation_date,effective_date,ticker,name,type
0,2015-12-28,2016-01-04,NaT,TW,Willis Towers Watson,ADDED
1,2015-12-28,2016-01-04,NaT,FOSL,Fossil Group,DELETED
2,2016-01-13,2016-01-15,NaT,EXR,Extra Space Storage,ADDED
3,2016-01-13,2016-01-15,NaT,CB,Chubb,DELETED
4,2016-01-22,2016-01-29,NaT,FRT,Federal Realty Trust,ADDED
...,...,...,...,...,...,...
151,2020-02-27,NaT,2020-03-03,XEC,Cimarex Energy,DELETED
152,2020-03-31,NaT,2020-04-03,OTIS,Otis Worldwide,ADDED
153,2020-03-31,NaT,2020-04-03,CARR,Carrier Global,ADDED
154,2020-03-31,NaT,2020-04-06,RTN,Raytheon,DELETED


In [24]:
outer_join = sp500_history1.merge(sp500_history2, how = 'outer', on=['ticker', 'type'], indicator=True)

In [25]:
outer_join.sort_values(['ticker', 'type'], inplace = True)

In [26]:
outer_join.to_excel('outer_join.xlsx')

In [47]:
# find entries where wiki dates are wrong
mask = (outer_join['_merge']=='both') & (-pd.Series.isna(outer_join['effective_date_y'])) & (outer_join['effective_date_x'] != outer_join['effective_date_y'])
outer_join_masked = outer_join.loc[mask]

In [49]:
outer_join_masked

,effective_date_x,ticker,wiki_name,type,announcement_date,implementation_date,effective_date_y,name,_merge
100,2017-08-08,BHF,Brighthouse Financial Inc,ADDED,2017-07-31,NaT,2017-08-07,Brighthouse Financial,both
164,2019-04-02,BHF,Brighthouse Financial,DELETED,2019-03-26,NaT,2019-04-03,Brighthouse Financial,both
166,2019-06-03,FLR,Fluor Corporation,DELETED,2019-05-28,NaT,2019-06-04,Fluor,both
138,2018-09-14,WCG,WellCare,ADDED,2018-09-11,NaT,2018-09-17,WellCare Health Plans,both
137,2018-09-14,XL,XL Group,DELETED,2018-09-11,NaT,2018-09-17,XL Group,both
